In [56]:
import pandas as pd
import numpy as np
import csv
import re
import datetime
from multiprocessing.dummy import Pool as ThreadPool
import pprint
from tqdm import tqdm
tqdm.pandas(desc="progress")

In [2]:
exclusion_terms = pd.read_excel("nanotech/exclusion_terms1.xlsx", header = None)
exclusion_terms =[str(i) for i in exclusion_terms[0].tolist()]

In [3]:
with open("nanotech/nano_organism_to_exclude.txt", "r") as myfile:
    nano_organism_to_exclude = []
    for line in myfile:
        line = line.strip("\n").split(";")
        for item in line:
            if item != "":
                nano_organism_to_exclude.append(item.strip())
#if any of these terms appear the patent should be excluded
exclusion_terms = exclusion_terms + nano_organism_to_exclude

In [4]:
measure_terms = pd.read_excel("nanotech/measure_terms.xlsx", header = None)
measures =[str(i) for i in measure_terms[0].tolist()]

In [5]:
with open("nanotech/MolEnv_1_in.txt", "r") as myfile:
    for line in myfile:
        Mol1 = line.strip("\n")
with open("nanotech/MolEnv_R_in.txt", "r") as myfile:
    for line in myfile:
        MolR = line.strip("\n")

In [6]:
conditions = pd.read_excel("nanotech/search_terms.xlsx")

In [7]:
def organize_text(list_of_terms, islist = False):
    terms = list_of_terms
    if islist:
        pass
    else:
        terms = terms.split(" OR ")
    terms = [str(item).strip('"') for item in terms]
    #there are no middle wild cards so just remove
    terms = [str(item).strip('*') for item in terms]
    terms = [re.sub('\*',"[a-zA-Z]*", item) for item in terms] #internal wildcards replace with regex to match letters
    terms = [r"\b"+item for item in terms]
    terms = "|".join(terms)
    terms = re.compile(terms,re.IGNORECASE)
    return terms

In [38]:
nano = r"\bnano[^\s]*"
nano_pattern = re.compile(nano)

In [9]:
measures = organize_text(measures, islist = True)
quantum_terms = organize_text(conditions.iloc[1,0])
self_terms = organize_text(conditions.iloc[2,0])
MolEnv1 = organize_text(Mol1)
MolEnvR = organize_text(MolR)
motor = organize_text(conditions.iloc[3,0])
micro = organize_text(conditions.iloc[4,0])
quasi = organize_text(conditions.iloc[5,0])
biosensor = organize_text(conditions.iloc[6,0])
exclusion_terms = organize_text(exclusion_terms, islist = True)

In [109]:
def search_for_terms(row, nano_pattern, measures, quantum_terms, self_terms, motor, micro,
                     quasi, biosensor,MolEnv1,MolEnvR, exclusion_terms):
    match_id = {"Nano Term":False,"Quantum Term":False,"Self Term":False,"Molecular Motor Term":False,"Micro Term":False,
                "Quasi Term":False,"Biosensor Term":False,"Biosensor 2 Term":False,"exclusion":False, 
                "measure_exclusion":False, "selection":False}
    
    nano_terms = re.findall(nano_pattern, row.text)
    are_measures = [bool(re.search(measures, item)) for item in nano_terms]
    final_inclusion=False
    if bool(re.search(quantum_terms, row.text)):
        match_id["Quantum Term"]=True
        final_inclusion=True
    if (bool(re.search(self_terms, row.text)) and bool(re.search(MolEnv1, row.text))):
        match_id["Self Term"]=True
        final_inclusion=True
    if bool(re.search(motor, row.text)):
        match_id["Molecular Motor Term"]=True
        final_inclusion=True
    if (bool(re.search(micro, row.text)) and bool(re.search(MolEnvR, row.text))) :
        match_id["Micro Term"]=True
        final_inclusion=True
    if (bool(re.search(quasi, row.text)) and bool(re.search(MolEnv1, row.text))) :
        match_id["Quasi Term"]=True
        final_inclusion=True
    if (bool(re.search(biosensor, row.text)) and bool(re.search(MolEnvR, row.text))):
        match_id["Biosensor Term"]=True
        final_inclusion=True
    if (bool(re.search(biosensor, row.text)) and bool(re.search(MolEnvR, row.text))):
        match_id["Biosensor Term"]=True
        final_inclusion=True
    if bool(re.search(exclusion_terms, row.text)):
        match_id["exclusion"] = True
    if all(are_measures) and len(are_measures)>0:
        match_id["measure_exclusion"]=True
    elif len(are_measures) >0:
        match_id["Nano Term"]=True
        final_inclusion=True
    match_id["selection"]=final_inclusion and not match_id["exclusion"]
    return(pd.Series(match_id))

In [100]:
def patent_search(file,nano_pattern, measures, quantum_terms, self_terms, motor, micro,
                     quasi, biosensor,MolEnv1,MolEnvR, exclusion_terms):
    current_file_data=pd.read_csv(file, sep="\t")
    print(current_file_data.columns)
    current_file_results=current_file_data.join(current_file_data.progress_apply(search_for_terms, axis=1,args=(nano_pattern, measures, quantum_terms, self_terms, motor, micro,
                     quasi, biosensor,MolEnv1,MolEnvR, exclusion_terms)))
    current_file_results.to_csv("patents.csv",index=False, mode="a")

In [ ]:
import os
subdirs = os.listdir("Z:\\Science Policy Portfolio\\PatentsView IV\\Data\\Parsed_2002-2017")
paths = [os.path.join("Z:\\Science Policy Portfolio\\PatentsView IV\\Data\\Parsed_2002-2017", item) for item in subdirs]

In [ ]:
fields_to_search = ["brf_sum_text.csv", "detail_desc_text.csv", "claim.csv"]
files = []
for path in paths:
    for j in fields_to_search:
        files.append(os.path.join(path, j))
pprint.pprint(files)
pprint.pprint(datetime.datetime.now().time())

In [ ]:
for i in files:
    patent_search(i,nano_pattern, measures, quantum_terms, self_terms, motor, micro,
                     quasi, biosensor,MolEnv1,MolEnvR, exclusion_terms)
    break

In [110]:
patent_search("./nano_test_data.txt",nano_pattern, measures, quantum_terms, self_terms, motor, micro,
                     quasi, biosensor,MolEnv1,MolEnvR, exclusion_terms)

Index(['uuid', 'patent_id', 'text'], dtype='object')



progress:   0%|                                                                                                                                                         | 0/16 [00:00<?, ?it/s]
progress: 17it [00:00, 1062.50it/s]                                                                                                                                                            